## MGMTMSA 403 Homework 2: Portfolio Optimization


## Pre-processing step: Estimate expected returns and covariance

In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv
import pandas as pd

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))


# Question 1

## a)

* **Parameters**

\begin{align*}
i &: \text{assets } \\
j &: \text{assets excluding i } \\
r_i &: \text{expected return of asset } i \\
C_{ij} &: \text{covariance matrix of } i \text{ and } j \\
\end{align*}

* **Decision Variables**

\begin{align*}
ω_i &: \text{weight on asset } i \\
\end{align*}

* **Objective Function**

\begin{equation*}
\sum_{i=1}^{4}\sum_{j=1}^{4} ω_i ω_j C_{ij}
\end{equation*}

* **Constraints**

> Expected return should be at least 0.5%

\begin{equation*}
\sum_{i=1}^{4} ω_{i} r_{i} \geq 0.5%
\end{equation*}

> Weights add up to 1

\begin{equation*}
\sum_{i=1}^{4} ω_{i} = 1
\end{equation*}

> Non-negativity

\begin{equation*}
ω_{i} \geq 0, \quad i = 1,...,390
\end{equation*}

In [54]:
from gurobipy import *

# Initialize the model
mod1 = Model()

# Decision variables
w1 = mod1.addVars(390)

# Objective function: Minimize total under-allocation percentage
mod1.setObjective(sum(w1[i] * w1[j] * C[i][j] for i in tickind for j in tickind), GRB.MINIMIZE)

# Constraints
mod1.addConstr(sum(w1[i] * avg_return[i] for i in tickind) >= 0.005)

mod1.addConstr(sum(w1[i] for i in tickind) == 1)

for i in tickind:
    mod1.addConstr(w1[i] >= 0)


# Solve the model
mod1.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 390 columns and 12 nonzeros
Model fingerprint: 0x24944e33
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 386 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0

In [55]:
# Display results
if mod1.status == GRB.OPTIMAL:
    ws1 = [round(w1[i].x, 4) for i in tickind]
    df = pd.DataFrame({'Ticker': tick4, 'Weight': ws1})
    print(df)

print('\nOptimal risk:', mod1.objval)
print('Solver time:', mod1.Runtime, 'seconds')

  Ticker  Weight
0   MSFT  0.2371
1     GS  0.0259
2     PG  0.0000
3   SCHP  0.7370

Optimal risk: 0.00017749326422824256
Solver time: 0.049616098403930664 seconds


## b) 

In [56]:
from gurobipy import *

tickind_all = []
for t in tickers:
    tickind_all.append(tickers.index(t))
tickind_all

# Initialize the model
mod2 = Model()

# Decision variables
w2 = mod2.addVars(390)

# Objective function: Minimize total under-allocation percentage
mod2.setObjective(sum(w2[i] * w2[j] * C[i][j] for i in tickind_all for j in tickind_all), GRB.MINIMIZE)

# Constraints

mod2.addConstr(sum(w2[i] * avg_return[i] for i in tickind_all) >= 0.005)

mod2.addConstr(sum(w2[i] for i in tickind_all) == 1)

for i in tickind:
    mod2.addConstr(w2[i] >= 0)


# Solve the model
mod2.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 390 columns and 784 nonzeros
Model fingerprint: 0xb5ac7cab
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.03s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl    

In [57]:
# Display results
print('\nOptimal risk:', mod2.objval)
print('Solver time:', mod2.Runtime, 'seconds')


Optimal risk: 2.8791752514001842e-05
Solver time: 0.09494400024414062 seconds


## c)

* **Parameters**

\begin{align*}
i &: \text{assets } \\
j &: \text{assets excluding i } \\
r_i &: \text{expected return of asset } i \\
C_{ij} &: \text{covariance matrix of } i \text{ and } j \\
\end{align*}

* **Decision Variables**

\begin{align*}
ω_i &: \text{weight on asset } i \\
s_i &: \text{1 if asset } i \text{ is selected,} \quad \text{0 otherwise} \\
\end{align*}

* **Objective Function**

\begin{equation*}
\sum_{i=1}^{390}\sum_{j=1}^{390} ω_i ω_j C_{ij}
\end{equation*}

* **Constraints**

> Selects at most 4 of the 390 stocks

\begin{equation*}
\sum_{i=1}^{390} s_i \leq 4, \quad i = 1,...,390
\end{equation*}

> If i is selected, then weight of i is smaller than 1; If i is not selected, then weight of i is 0

\begin{equation*}
ω_{i} \leq s_{i}, \quad i = 1,...,390
\end{equation*}

> Expected return should be at least 0.5%

\begin{equation*}
\sum_{i=1}^{4} ω_{i} r_{i} \geq 0.5%
\end{equation*}

> Weights add up to 1

\begin{equation*}
\sum_{i=1}^{4} ω_{i} = 1
\end{equation*}

> Non-negativity

\begin{equation*}
ω_{i} \geq 0, \quad i = 1,...,390
\end{equation*}

In [80]:
from gurobipy import *

tickind_all = []
for t in tickers:
    tickind_all.append(tickers.index(t))
tickind_all

# Initialize the model
mod3 = Model()

# Decision variables
w3 = mod3.addVars(390)
s = mod3.addVars(390, vtype=GRB.BINARY, name="x")

# Objective function: Minimize total under-allocation percentage
mod3.setObjective(sum(w3[i] * w3[j] * C[i][j] for i in tickind_all for j in tickind_all), GRB.MINIMIZE)

# Constraints

mod3.addConstr(sum(s[i] for i in tickind_all) <=4)

mod3.addConstr(sum(w3[i] * avg_return[i] for i in tickind_all) >= 0.005)

mod3.addConstr(sum(w3[i] for i in tickind_all) == 1)

for i in tickind_all:
    mod3.addConstr(w3[i] <= s[i])

for i in tickind_all:
    mod3.addConstr(w3[i] >= 0)


# Solve the model
mod3.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x2525555f
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.04s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objec

In [81]:
# Display results
if mod3.status == GRB.OPTIMAL:
    print("Ticker\t\tWeight")
    for i in range(n):
        if s[i].x:
            print(tickers[i], '\t\t', w3[i].x)

print('\nSolver time:', mod3.Runtime)
print('Optimal value:', mod3.objval)

Ticker		Weight
CME 		 0.1264114192949047
LLY 		 0.07547619035437458
NVDA 		 0.043753706728431804
BND 		 0.754358683622289

Solver time: 29.408276081085205
Optimal value: 6.753470760728118e-05


# Question 2

## a)

The optimal risk in Model 1 is higher than Model 2. </br>

This is because Model 2 has access to a broader range of stock options, which helps with the diversification of stock assets.

## b)

The optimal risk in Model 2 is lower than Model 3. </br>

This is because Model 2 has access to a broader range of stock options, which helps with the diversification of stock assets. </br>
Also, risk is subject to a constraint on expected return. The more constraints there are, the higher the risk is. Model 3 has two more constraints and thus a higher risk than Model 2. </br>

# Question 3

## a)

In [82]:
from gurobipy import *

tickind_all = []
for t in tickers:
    tickind_all.append(tickers.index(t))
tickind_all

# Initialize the model
mod3a = Model()

# Decision variables
w3a = mod3a.addVars(390)
s3a = mod3a.addVars(390, vtype=GRB.BINARY, name="x")

# Objective function: Minimize total under-allocation percentage
mod3a.setObjective(sum(w3a[i] * w3a[j] * C[i][j] for i in tickind_all for j in tickind_all), GRB.MINIMIZE)

# Constraints

mod3a.addConstr(sum(s3a[i] for i in tickind_all) <=4)

mod3a.addConstr(sum(w3a[i] * avg_return[i] for i in tickind_all) >= 0.005)

mod3a.addConstr(sum(w3a[i] for i in tickind_all) == 1)

for i in tickind_all:
    mod3a.addConstr(w3a[i] <= s3a[i])

for i in tickind_all:
    mod3a.addConstr(w3a[i] >= 0)

    
# Terminate after 3 seconds
mod3a.Params.TimeLimit = 3.0

# Solve the model
mod3a.optimize()

Set parameter TimeLimit to value 3
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x2525555f
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.03s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 work units)

    Nodes 

In [83]:
print('Optimal value of terminated model:', mod3a.objval)
print('Optimal value of original model:', mod3.objval)

Optimal value of terminated model: 6.758574167459062e-05
Optimal value of original model: 6.753470760728118e-05


The objective function value at termination is the same as the optimal value obtained in question 1c.

## b)

In [84]:
from gurobipy import *

tickind_all = []
for t in tickers:
    tickind_all.append(tickers.index(t))
tickind_all

# Initialize the model
mod3b = Model()

# Decision variables
w3b = mod3b.addVars(390)
s3b = mod3b.addVars(390, vtype=GRB.BINARY, name="x")

# Objective function: Minimize total under-allocation percentage
mod3b.setObjective(sum(w3b[i] * w3b[j] * C[i][j] for i in tickind_all for j in tickind_all), GRB.MINIMIZE)

# Constraints

mod3b.addConstr(sum(s3b[i] for i in tickind_all) <=4)

mod3b.addConstr(sum(w3b[i] * avg_return[i] for i in tickind_all) >= 0.005)

mod3b.addConstr(sum(w3b[i] for i in tickind_all) == 1)

for i in tickind_all:
    mod3b.addConstr(w3b[i] <= s3b[i])

for i in tickind_all:
    mod3b.addConstr(w3b[i] >= 0)

    
# Terminate after reaching a gap of 10%
mod3b.Params.MIPGap = 0.1

# Solve the model
mod3b.optimize()

Set parameter MIPGap to value 0.1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x2525555f
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.03s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 work units)

    Nodes  

In [85]:
print('Solver time of terminated model:', mod3a.Runtime)
print('Solver time of original model:', mod3.Runtime)

Solver time of terminated model: 3.11930513381958
Solver time of original model: 29.408276081085205


The solver time is shorter than the solution time obtained in question 1c.